# LSTM / GRU aplicados a señales ECG (EKG)

## I. Teoría breve del tópico elegido

Las señales ECG (electrocardiográficas) representan la actividad eléctrica del corazón como una **secuencia temporal** [3].   
Por ello, son especialmente adecuadas para modelos que pueden **recordar información del pasado** y relacionarla con el presente [1][2].

Las redes neuronales recurrentes (RNN tradicionales) fueron diseñadas para este propósito, pero sufren de problemas como:

- **Vanishing gradient** (la red “olvida” señales antiguas) 
- Dificultad para aprender dependencias largas
- Poca robustez ante señales ruidosas

Para resolver estos problemas surgieron **LSTM** y **GRU**, que se convirtieron en los modelos dominantes en tareas de ECG [1][2][3].

---

## 🧠 ¿Qué es LSTM?

Las **Long Short-Term Memory (LSTM)** son un tipo avanzado de RNN que incorporan un mecanismo interno de memoria [1], permitiendo:

- Recordar información a largo plazo  
- Controlar qué información entra y sale  
- Evitar que los gradientes se desvanezcan  

Una celda LSTM tiene:

### ✔ Input gate  
Decide cuánta información nueva se debe incorporar [1].

### ✔ Forget gate  
Determina cuánta información previa debe olvidarse. Esto es crucial en ECG, donde cada latido tiene estructura similar pero puede contener ruido [3].

### ✔ Output gate  
Controla qué información se envía a la siguiente capa [1].

### ✔ Cell state (memoria a largo plazo)  
Transporta información clave sobre la señal cardíaca [1].

---

## 🔁 ¿Qué es GRU?

Las **Gated Recurrent Units (GRU)** simplifican la estructura de las LSTM manteniendo un desempeño similar.  
Tienen solo dos puertas:

### ✔ Update gate  
Combina input y forget; controla cuánta memoria pasada mantener.

### ✔ Reset gate  
Define cuándo olvidar información pasada, útil para eventos abruptos en ECG (p.ej., latidos prematuros).

Las GRU son más ligeras, entrenan más rápido y consumen menos recursos—ideal para dispositivos embebidos.

---

## ❤️ ¿Por qué LSTM/GRU funcionan tan bien en señales ECG?

El ECG tiene patrones temporales complejos [3]:

- Ritmo cardíaco (intervalos RR)
- Morfología P-QRS-T
- Variabilidad cardíaca
- Anomalías específicas de arritmias
- Señal de identidad biométrica

LSTM y GRU pueden:

✔ Capturar dependencias largas entre latidos 
✔ Aprender la morfología completa sin extracción manual 
✔ Ser robustas a ruido y artefactos del ECG   
✔ Trabajar con secuencias cortas o largas  
✔ Entrenar modelos end-to-end 
---

# II. Papers seleccionados sobre LSTM/GRU aplicados a ECG

A continuación, se explican **tres papers**, resaltando especialmente **cómo implementan LSTM o GRU** y **por qué funcionan bien en su aplicación**.

---

# 📘 **PAPER 1: Kim & Pyun (2020) — BiLSTM + Late Fusion para identificación ECG** [4]

**Modelo:** Deep Recurrent Neural Network con **3 capas BiLSTM**  
**Problema:** Autenticación biométrica de personas usando ECG  
**Por qué usan BiLSTM:**  
- El ECG contiene información que depende del pasado y ligeramente del futuro (pico T influenciado por QRS anterior)  
- Una LSTM bidireccional analiza la señal “hacia adelante y hacia atrás”, enriqueciendo la representación  

---

## 🧩 Explicación ampliada del uso de LSTM en este paper

### 1. Preprocesamiento  
Los autores aplican:

- Filtro derivativo para resaltar QRS  
- Media móvil para suavizar  
- Normalización min-max  
- Segmentación por latido usando R-peaks  

Esto permite que la LSTM reciba secuencias “limpias”.

---

### 2. Arquitectura BiLSTM más detallada

Cada capa BiLSTM procesa:

- Secuencia forward → aprende progresión temporal normal del latido  
- Secuencia backward → aprende simetrías, fin de onda T, retorno a baseline  

La salida de ambas direcciones se combina, lo cual permite:

- Mayor discriminación entre sujetos  
- Aprendizaje robusto ante variabilidad intra-individuo  
- No requiere extracción de características manuales  

---

### 3. Late Fusion  
El modelo produce una predicción por timestep y luego realiza un **promedio ponderado**.  
Esto:

- Suaviza variaciones inesperadas  
- Aumenta la estabilidad de la clasificación  
- Reduce el impacto de ruido o latidos atípicos  

---

### 4. Resultados  
- **100% accuracy** en NSRDB  
- **99.8% accuracy** en MITDB  
- Mejores resultados que CNN, SVM y RNN tradicionales  

---

### ✔ En este paper, el LSTM funciona porque:
- Captura *patrones personales únicos* del ECG  
- Modela relaciones temporales finas entre latidos  
- Puede trabajar con secuencias cortas (muy útil en biometría real-time)

---

# 📘 **PAPER 2: Satheeswaran et al. (2024) — LSTM para clasificación de arritmias** [5]

**Modelo:** LSTM clásico dentro de una arquitectura RNN  
**Problema:** Clasificación de arritmias del MIT-BIH  
**Por qué usan LSTM:** Las anormalidades cardíacas se manifiestan como alteraciones en los intervalos y morfología temporal.

---

## 🧩 Explicación ampliada del uso de LSTM en este paper

### 1. Preprocesamiento  
Incluye:

- Filtros de ruido  
- Normalización  
- Segmentación basada en ventana  
- Preparación de latidos independientes  

La LSTM recibe entradas de longitudes estandarizadas.

---

### 2. La LSTM como núcleo secuencial

La LSTM aprende:

- Variación en intervalos RR (indicador clave de arritmias)  
- Cambios en duración QRS  
- Aparición de latidos ventriculares prematuros  
- Morfología P anómala  

La capacidad de memoria permite distinguir:

- Latidos **normales**  
- Latidos **supraventriculares anormales**  
- Latidos **ventriculares**  
- Latidos peligrosos como R-on-T  

---

### 3. Resultados  
- **98–99% accuracy**  
- Excelente recall en clases minoritarias  
- LSTM > RNN simple  

---

### ✔ En este paper, el LSTM funciona porque:
- Aprende dependencias temporales largas (comportamiento entre latidos sucesivos)  
- Reduce necesidad de feature engineering  
- Puede adaptarse a ruido y señales incompletas  

---

# 📘 **PAPER 3: Yun Ju et al. (2019) — Deep Bidirectional GRU (DBGRU)** [6]

**Modelo:** 6 capas de **Bidirectional GRU**  
**Problema:** Clasificar **23 tipos** de arritmias (tarea muy difícil)  
**Por qué usan GRU bidireccional:**  
- GRU es más ligero que LSTM → se pueden usar más capas profundas  
- Permite analizar secuencias largas sin costos computacionales excesivos  

---

## 🧩 Explicación ampliada del uso de GRU en este paper

### 1. PCA como preprocesamiento  
Reduce ruido y dimensionalidad → mejora entrada a la GRU.  
Esto es importante porque señales MIT-BIH contienen artefactos.

---

### 2. Arquitectura GRU de 6 capas

- 3 capas GRU bidireccional de 256 unidades  
- 3 capas GRU bidireccional de 128 unidades  
- Dropout para evitar overfitting  
- Fully connected + Softmax  

La GRU captura:

- Dependencias temporales largas  
- Cambios en secuencia P-QRS-T  
- Anomalías específicas de las 23 clases  

---

### 3. Ventaja clave de GRU  
Al ser más simples que LSTM:

- Entrenan más rápido  
- Se pueden apilar más capas  
- Previenen overfitting por exceso de parámetros  
- Funcionan mejor en datasets grandes o con secuencias largas  

---

### 4. Resultados  
- **99.51% accuracy** (train)  
- **97.86% accuracy** (test)  
- Superó a:
  - Unidirectional LSTM  
  - Unidirectional GRU  
  - CNN  
  - DNN  

---

### ✔ En este paper, GRU funciona porque:
- Maneja mejor secuencias largas multiclase  
- Es más eficiente que LSTM para modelos profundos  
- Capta patrones temporales relevantes con menos complejidad  

---

# III.🩺 Repositorios de GitHub que utilizan LSTM/GRU y son aplicados a ECG

# ## *1. Notebook LSTM — Clasificación de Señales ECG*
Link github: https://github.com/AlTheMan/ECG-Classifier-LSTM
DATASET:  https://www.nature.com/articles/s41597-020-0495-6

### *🎯 Objetivo*

Entrenar un modelo basado en *LSTM* para clasificar señales ECG, aprendiendo automáticamente patrones temporales y morfológicos del corazón sin necesidad de extracción manual de características.

### *📂 Base de datos utilizada*

El notebook carga un dataset que contiene:

•⁠  ⁠Señales ECG segmentadas en *ventanas temporales*.
•⁠  ⁠Cada ventana representa un *latido* o un fragmento de 1–3 latidos.
•⁠  ⁠Las muestras están en formato:

  
⁠   (timesteps, 1) → señal ECG unidimensional
   ⁠
•⁠  ⁠Las etiquetas corresponden a clases de arritmias o identificación de individuos.

La data se divide en:

•⁠  ⁠*Entrenamiento (train)*
•⁠  ⁠*Validación (dev)*
•⁠  ⁠*Prueba (test)*


### * Procesamiento de datos*

El notebook realiza:

•⁠  ⁠*Normalización* de amplitud
•⁠  ⁠*Segmentación en ventanas* (fijas o basadas en picos R)
•⁠  ⁠Reestructuración a formato requerido por LSTM:


(samples, timesteps, features=1)



### *🧠 Arquitectura LSTM*

El modelo se basa en:

•⁠  ⁠1 o más capas *LSTM* para capturar dependencias temporales largas
•⁠  ⁠Capas densas finales para la clasificación
•⁠  ⁠Entrenamiento con *Adam* y función de pérdida *categorical_crossentropy*

El LSTM analiza la señal muestra por muestra, aprendiendo patrones como:

•⁠  ⁠forma del complejo QRS
•⁠  ⁠ondas P y T
•⁠  ⁠intervalos temporales entre latidos (RR)


### *📊 Resultado del notebook*

•⁠  ⁠Entrenamiento completo con seguimiento de métricas
•⁠  ⁠Gráficas de pérdida y precisión
•⁠  ⁠Evaluación en test
•⁠  ⁠Modelo final entrenado para clasificación ECG


# ## *2. Script GRU (⁠ gru.py ⁠) — Clasificación Multietiqueta de ECG*
link github: https://github.com/HemaxiN/DL_ECG_Classification/blob/main/cnn_gru.py
DATASET: https://physionet.org/content/ptb-xl/1.0.1/
### *🎯 Objetivo*

Entrenar un modelo *GRU* para clasificación *multietiqueta* (4 clases) utilizando señales ECG multicanal (*3 derivaciones*), incluyendo optimización de umbrales y métricas clínicas reales (sensibilidad/especificidad).



## *📂 Base de datos utilizada*

Cargada mediante ⁠ Dataset_for_RNN ⁠:

•⁠  ⁠Cada muestra tiene forma:


X → (1000 timesteps, 3 features)
y → (4 etiquetas binarias)


•⁠  ⁠Dataset dividido en:

  * *Train* (batch_size=512)
  * *Validation*
  * *Validation para thresholds*
  * *Test*

Las clases están desbalanceadas, por ello se calculan *pesos por clase*.


## *Procesamiento de datos*

La data ya viene preprocesada desde la carpeta ⁠ data_for_rnn ⁠, pero el script realiza:

•⁠  ⁠Reorganización de tensores para secuencias GRU
•⁠  ⁠Carga por batches
•⁠  ⁠Aplicación de ⁠ class_weights ⁠
•⁠  ⁠Preparación para clasificación multietiqueta con BCEWithLogitsLoss


## *🧠 Arquitectura GRU*

El modelo implementa:

⁠ python
nn.GRU(input_size=3,
       hidden_size=128,
       num_layers=2,
       dropout=0.3,
       batch_first=True,
       bidirectional=True/False)
 ⁠

Salida final:

•⁠  ⁠Se usa el último timestep (o combinación bidireccional)
•⁠  ⁠Pasa a ⁠ Linear(hidden_size * d → 4) ⁠
•⁠  ⁠Activación sigmoide durante predicción

El GRU aprende relaciones temporales en las derivaciones del ECG, pero con menor complejidad que LSTM.

---

## * Entrenamiento*

Incluye:

•⁠  ⁠Optimización con Adam o SGD
•⁠  ⁠Early stopping
•⁠  ⁠Guardado del mejor modelo según validación
•⁠  ⁠Cálculo de pérdida por epoch

---

## *🔎 Optimización de umbrales*

El script calcula un umbral óptimo por clase usando:

•⁠  ⁠Curva ROC
•⁠  ⁠G-Mean = √(TPR × (1 – FPR))
•⁠  ⁠Threshold por clase para mejorar:

  * Sensibilidad
  * Especificidad

---

## *📊 Evaluación final*

El modelo entrega:

•⁠  ⁠Matriz (TP, FN, FP, TN) por clase
•⁠  ⁠Sensibilidad por enfermedad
•⁠  ⁠Especificidad por enfermedad
•⁠  ⁠AUROC por clase
•⁠  ⁠Resultados guardados en archivo ⁠ .txt ⁠

---

# ## *Resumen General*

•⁠  ⁠*LSTM Notebook* → Clasificación basada en morfología ECG, un enfoque más clásico, ideal para secuencias largas y dependencias complejas.
•⁠  ⁠*GRU Script* → Clasificación multietiqueta avanzada, optimizada, con umbrales personalizados y enfoque multicanal. GRU ofrece menor costo computacional con rendimiento comparable.
---

# ✔ Conclusión general

Los tres papers demuestran:

- **LSTM → mejor para biometría y análisis fino de morfología temporal**  
- **GRU → mejor para modelos profundos o secuencias largas con múltiples clases**  
- Ambos modelos pueden superar 97–100% de precisión  
- Son adecuados para aplicaciones reales: biometría, diagnóstico y monitoreo en tiempo real  


---
# Referencias:

[1] S. Hochreiter and J. Schmidhuber, “Long Short-Term Memory,” Neural Computation, vol. 9, no. 8, pp. 1735–1780, 1997. Disponible en: https://www.bioinf.jku.at/publications/older/2604.pdf
[2] K. Cho et al., “Learning Phrase Representations using RNN Encoder–Decoder for Statistical Machine Translation,” in Proc. EMNLP, 2014, pp. 1724–1734. Disponible en: https://arxiv.org/abs/1406.1078
[3] I. Goodfellow, Y. Bengio, and A. Courville, *Deep Learning*.MIT Press, 2016.  
Disponible en: https://www.deeplearningbook.org/
[4] B.-H. Kim and J.-Y. Pyun, “ECG Identification for Personal Authentication Using LSTM-Based Deep Recurrent Neural Networks,” Sensors, vol. 20, no. 11, pp. 3069, May 2020. Disponible en:https://www.mdpi.com/1424-8220/20/11/3069
[5] V. Satheeswaran, A. A. Al-Shargabi, S. Alshargabi, and M. Al-Turjman, “Deep Learning Based Classification of ECG Signals Using RNN-LSTM Mechanism,” Journal of Electrical Engineering, Electronic Control Management and Information Technology, vol. 6, no. 2, pp. 1–12, 2024. Disponible en: https://jeeemi.org/index.php/jeeemi/article/view/496
[6] Y. Ju, M. Zhang, and H. Zhu, “Study on a New Deep Bidirectional GRU Network for Electrocardiogram Signals Classification,” in Proc. 3rd Int. Conf. Comput. Eng., Inf. Sci. Appl. Technol. (ICCIA), 2019, pp. 355–359. Disponible en (PDF):https://www.researchgate.net/publication/334658737_Study_on_a_New_Deep_Bidirectional_GRU_Network_for_Electrocardiogram_Signals_Classification 

